In [ ]:
import os
import pickle
import pandas as pd
from collections import defaultdict

In [ ]:
#symptomatic, location

In [ ]:
questions=["age","tipo","tecnica","family","history","symtomatic",
           "prosthesis","birads","density_mammo","calcifications_benign","density_echo","ganglio_mamo","lymph_benign","lymph_suspicious","parenchymal_distortion","simple_cyst","ductal_ectasia",
          "nodules_echo_num","nodules_echo_description","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_location","nodules_echo_size","nodules_echo_known","nodules_echo_stable"]
questions=["tipo","tecnica","family",
           "prosthesis","birads","density_mammo","calcifications_benign","ganglio_mamo","density_echo","lymph_benign","lymph_suspicious","simple_cyst","ductal_ectasia","nodules_echo_num","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_known","nodules_echo_stable"]


In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
def calculate_f1(pred,truth,point=False,average='macro'):
    pred_clean={}
    truth_clean={}
    for key,real in truth.items():
        if key in pred:
            
            predicted=pred[key] 
            if point==True:
                predicted=predicted.split(".")[0]+"."
            elif point=="mm":
                if "unknown" in predicted:
                    predicted=predicted.split(".")[0]+"."
                else:
                    predicted=predicted.split("mm")[0]+"mm."
                
            pred_clean[key]=predicted.lower()
            truth_clean[key]=str(real).lower()
    # Get ground truth and predicted labels
    y_true = list(truth_clean.values())
    y_pred = list(pred_clean.values())
    
    # Define the valid labels based on the truth only
    valid_labels = sorted(set(y_true))
    
    # Replace out-of-scope predictions with something invalid
    # e.g., keep them but let them be counted as wrong
    y_pred_cleaned = [
        p if p in valid_labels else 'INVALID' for p in y_pred
    ]
    
    # Now align with true values
    y_true_final = []
    y_pred_final = []
    
    for yt, yp in zip(y_true, y_pred_cleaned):
        y_true_final.append(yt)
        y_pred_final.append(yp)
    
    # Compute F1 over valid labels only
    f1 = f1_score(
        y_true_final,
        y_pred_final,
        labels=valid_labels,   # Only ground-truth classes
        average='macro'        # or whatever average you want
    )
    report = classification_report(
        y_true,
        y_pred_cleaned,
        labels=valid_labels,     # Only ground truth classes
        zero_division=0,         # Avoid warnings for undefined precision/recall
        digits=4                 # Optional: better precision
    )
    print(report)
    return f1


In [ ]:
def calculate_acc(pred,truth,point=False):
    acc=0
    total=0
    for key,real in truth.items():
        if key in pred:
            total+=1
            predicted=pred[key]    
            if point==True:

                predicted=predicted.split(".")[0]+"."

            elif point=="mm":
                if "unknown" in predicted:
                    predicted=predicted.split(".")[0]+"."
                else:
                    predicted=predicted.split("mm")[0]+"mm."
            if predicted.lower()==str(real).lower():
                acc+=1
            else:
                print(key,real,predicted)
        else:
            print(key,real)
        # else:
        #     print(key)
    return acc/total

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
accuracies=defaultdict(list)
f1s=defaultdict(list)
models=["biogpt","clinicalt5","clinicalt5_tokenized", "biogpt_no_info","biogpt_answer"]
for model in models:
    print(model)
    accuracies["model"].append(model)
    f1s["model"].append(model)
    for tipo in questions:
        
        print(tipo)
        with open(f"test_results/truth_dic/{tipo}.pkl", "rb") as file:
                truth=pickle.load(file)
        with open(f"test_results/results_dic_{tipo}/{model}.pkl", "rb") as file:
                output=pickle.load(file)
        if tipo=="family":
            print(len(output))
        if tipo=="birads":
            truth["746-766-264-20240605-182548_birads"]="BI-RADS 2."
        
        if tipo!="nodules_echo_size":
            acc=calculate_acc(output,truth,point=True)
            
            f1=calculate_f1(output,truth,point=True)
            if tipo=="density_echo":
                print(truth,output)
                print(f1)
                print(len(truth),len(output))
        else:
            acc=calculate_acc(output,truth,point="mm")
            f1=calculate_f1(output,truth,point="mm")
        accuracies[tipo].append(acc)
        f1s[tipo].append(f1)



In [ ]:
data=pd.DataFrame(accuracies)
data=data.set_index("model")
data['average'] = data.mean(axis=1)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

In [ ]:
biogpt_acc = list(data.loc["biogpt",:])[:-1]
biogpt_acc

In [ ]:
data=pd.DataFrame(f1s)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

In [ ]:
biogpt_f1 = list(data.loc["biogpt",:])[:-1]

In [ ]:
TIPO=["biopsy report", "nodal staging ultrasound report", "normal control or revision report"]
TECNICA=["only ultrasound study", "only mammography study", "mammography and ultrasound"]
FAMILY=["no family history", "first degree", "second degree"]
PROSTHESIS=["no prosthesis","yes prosthesis"]
BIRADS=["BI-RADS 0","BI-RADS 1","BI-RADS 2","BI-RADS 3","BI-RADS 4A","BI-RADS 4B","BI-RADS 4C","BI-RADS 5","BI-RADS 6"]
DENSITY_MAMMO=["ACR A","ACR B","ACR C","ACR D","unknown density mammo"]
CALCIFICATIONS_BENIGN=["no calcifications","yes calcifications"]
GANGLIO_MAMO=["no ganglio","yes ganglio"]
DENSITY_ECHO=["homogeneous fibroglandular","heterogeneous fibroglandular","fibroglandular and fat","homogeneous fatty","unknown density echo"]
LYMPH_BENIGN=["no lymph benign","yes lymph benign"]
LYMPH_SUSPICIOUS=["no lymph suspicious","yes lymph suspicious"]
SIMPLE_CYST=["no cyst","yes cyst"]
DUCTAL_ECTASIA=["no ectasia","yes ectasia"]
NODULES_ECHO=["no nodules", "yes nodules"]
NODULES_SHAPE=["oval","round","lobulated","irregular","unknown shape"]
NODULES_MARGIN=["circumscribed","spiculated","indistinct","not circumscribed","unknown margin"]
NODULES_ECHOGENICITY=["hypoechoic", "isoechoic", "heterogeneous","complex cystic and solid","unknown echogenicity"]
NODULES_KNOWN=["no known", "yes known"]
NODULES_STABLE=["grown stable","shrunk stable", "yes stable"]

DICTIONARY={"tipo":TIPO,"tecnica":TECNICA,"family":FAMILY,"prosthesis":PROSTHESIS,"birads":BIRADS,"density_mammo":DENSITY_MAMMO,"calcifications_benign":CALCIFICATIONS_BENIGN,
            "ganglio_mamo":GANGLIO_MAMO,"density_echo":DENSITY_ECHO,"lymph_benign":LYMPH_BENIGN,"lymph_suspicious":LYMPH_SUSPICIOUS,"simple_cyst":SIMPLE_CYST,"ductal_ectasia":DUCTAL_ECTASIA,
           "nodules_echo": NODULES_ECHO,"nodules_shape":NODULES_SHAPE,"nodules_margin":NODULES_MARGIN, "nodules_echogenicity":NODULES_ECHOGENICITY, "nodules_known":NODULES_KNOWN, "nodules_stable":NODULES_STABLE}

def evaluate_per_question(predicted, tested, DICTIONARY, average="macro"):
    """
    Evaluate model predictions per question type.
    
    Parameters:
    - predicted: array of predicted label indices (flattened from all folds)
    - tested: array of true label indices (same shape as predicted)
    - DICTIONARY: dict mapping each question to its list of class names
    """
    
    # Step 1: Build global index → (question, class_name) mapping
    idx_to_question_value = {}
    offset = 0
    question_offsets = {}
    for question, class_list in DICTIONARY.items():
        question_offsets[question] = offset
        for i, label in enumerate(class_list):
            idx_to_question_value[offset + i] = (question, label)
        offset += len(class_list)

    # Step 2: Group predictions by question
    per_question_true = defaultdict(list)
    per_question_pred = defaultdict(list)

    for true_idx, pred_idx in zip(tested, predicted):
        q_true, _ = idx_to_question_value[true_idx]
        # You can check if q_true == q_pred here for safety if needed
        per_question_true[q_true].append(int(true_idx))
        per_question_pred[q_true].append(int(pred_idx))

    # Step 3: Classification reports

    f1s={}
    accuracies={}
    for question, true_labels in per_question_true.items():
        pred_labels = per_question_pred[question]
        label_names = DICTIONARY[question]
        start = question_offsets[question]
        end = start + len(label_names)
        question_label_ids = list(range(start, end))
        # print(question)
        #Añadimos el family history third que los BERT modelos no pueden capturar
        print(len(true_labels))
        if question=="family":
            print(len(true_labels))
            true_labels=[200]+true_labels
            pred_labels=[7]+pred_labels
            print(len(true_labels))
        
        f1=f1_score(true_labels,pred_labels,labels=sorted(set(true_labels)),average=average)
        f1s[question]=f1
        accuracy=accuracy_score(true_labels,pred_labels)
        accuracies[question]=accuracy
    return f1s,accuracies        


In [ ]:
import re
import numpy as np
models=["predicted_biobert","predicted_biomed","predicted_bluebert"]
accuracies=defaultdict(list)
f1s=defaultdict(list)
for model in models:
    model_truth=re.sub("predicted","tested",model)
    truth=np.load(f"test_results/{model_truth}_classification.npy")
    outputs=np.load(f"test_results/{model}_classification.npy")
    print(truth)
    
    f1s_model,accuracies_model=evaluate_per_question(outputs,truth,DICTIONARY)
    accuracies["model"].append(model)
    f1s["model"].append(model)
    for question in DICTIONARY.keys():
        accuracies[question].append(accuracies_model[question])
        f1s[question].append(f1s_model[question])
        

In [ ]:
data=pd.DataFrame(f1s)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

In [ ]:
biomed_f1 = list(data.loc["predicted_biomed",:])[:-1]

In [ ]:
data=pd.DataFrame(accuracies)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

In [ ]:
biomed_acc = list(data.loc["predicted_biomed",:])[:-1]

In [ ]:
from scipy.stats import wilcoxon


# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(biogpt_acc, biomed_acc, alternative='greater')

print(f"Wilcoxon statistic: {statistic}")
print(f"p-value: {p_value}")

# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(biogpt_f1, biomed_f1, alternative='greater')

print(f"Wilcoxon statistic f1: {statistic}")
print(f"p-value: {p_value}")

In [ ]:
biogpt=[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1677,
 1.0,
 1.0,
 0.0019,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2638,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8763,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.3209,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4092,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8641,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5016,
 0.1649,
 1.0,
 1.0,
 1.0,
 0.252,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0623,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9951,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8368,
 1.0,
 0.3967,
 1.0,
 1.0,
 0.9867,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.489]

In [ ]:
biogpt_no_age=[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1677,
 1.0,
 1.0,
 0.0019,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2638,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8763,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.3209,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4092,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8641,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5016,
 0.1649,
 1.0,
 1.0,
 1.0,
 0.252,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0623,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9951,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8368,
 1.0,
 0.3967,
 1.0,
 1.0,
 0.9867,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.489]

In [ ]:
BioMedBERT=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7013, 1.0, 1.0, 1.0, 0.1352, 0.1352, 1.0, -0.0418, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0964, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7602, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4887, 1.0, 0.1503, 1.0, 1.0, 1.0, 1.0, 0.7231, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8368, 1.0, 0.3967, 1.0, 1.0, 0.991, 1.0, 1.0, 0.5678, 1.0, 1.0, 1.0, 1.0]

In [ ]:
BioMedBERT_no_age=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7013, 1.0, 1.0, 1.0, 0.1352, 0.1352, 1.0, -0.0418, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0964, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7602, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4887, 1.0, 0.1503, 1.0, 1.0, 1.0, 1.0, 0.7231, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8368, 1.0, 0.3967, 1.0, 1.0, 0.991, 1.0, 1.0, 0.5678, 1.0, 1.0, 1.0, 1.0]

In [ ]:
from scipy.stats import wilcoxon


# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(BioMedBERT,biogpt, alternative='greater')

print(f"Wilcoxon statistic: {statistic}")
print(f"p-value: {p_value}")

# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(BioMedBERT_no_age,biogpt_no_age, alternative='greater')

print(f"Wilcoxon statistic f1: {statistic}")
print(f"p-value: {p_value}")